In [1]:
import sys
from pathlib import Path
import os

repo_root_dir: Path = Path().resolve().parent
sys.path.append(str(repo_root_dir))
repo_root_dir

WindowsPath('C:/Users/chris/Desktop/GitHub/air-quiality-vs-illness')

In [2]:
import requests
import pandas as pd
from pprint import pprint
import pycountry

from src import openaq_api

In [3]:
OPENAQ_API_KEY = "2df1e1ad492dbcc9293e82253f0ddffca796f334d424d0c1765e827c87a219ea"

BASE_URL = "https://api.openaq.org/v3/"

In [4]:
headers = {
    "X-API-Key": OPENAQ_API_KEY
}

In [5]:
endpoint_parameters = f"{BASE_URL}parameters"
endpoint_locations = f"{BASE_URL}locations"
endpoint_countries = f"{BASE_URL}countries"

In [6]:
major_us_cities = [
    "Los Angeles", "New York", "Chicago", "Houston", "Phoenix",
    "Philadelphia", "San Antonio", "San Diego", "Dallas", "San Jose"
]


In [7]:
parameters_params = {
    "limit": 10,
}

In [8]:
params = {"countries_id": [79], "limit": 100}

response = requests.get(endpoint_locations, params=params, headers=headers)

locations = response.json()
pprint(locations)

{'meta': {'found': '>100',
          'limit': 100,
          'name': 'openaq-api',
          'page': 1,
          'website': '/'},
 'results': [{'bounds': [-0.06821799979396657,
                         51.59929999967941,
                         -0.06821799979396657,
                         51.59929999967941],
              'coordinates': {'latitude': 51.59929999967941,
                              'longitude': -0.06821799979396657},
              'country': {'code': 'GB', 'id': 79, 'name': 'United Kingdom'},
              'datetimeFirst': {'local': '2016-01-30T01:00:00Z',
                                'utc': '2016-01-30T01:00:00Z'},
              'datetimeLast': {'local': '2025-05-21T18:00:00+01:00',
                               'utc': '2025-05-21T17:00:00Z'},
              'distance': None,
              'id': 60,
              'instruments': [{'id': 2, 'name': 'Government Monitor'}],
              'isMobile': False,
              'isMonitor': True,
              'licenses': [

In [ ]:
location_id = 2416

params = {"limit": 10}

response = requests.get(f"{endpoint_locations}/{location_id}/sensors", params=params, headers=headers)

pprint(response.json())

In [8]:
country_names = ["Norway", "United Kingdom", "Japan"]

iso = openaq_api.get_iso_country_codes(country_names)
iso

['NO', 'GB', 'JP']

In [10]:


locations = openaq_api.get_location_ids(iso)

In [18]:
locations["NO"]

[{'id': 2947,
  'name': 'Leiret',
  'locality': 'Elverum',
  'timezone': 'Europe/Oslo',
  'country': {'id': 53, 'code': 'NO', 'name': 'Norway'},
  'owner': {'id': 4, 'name': 'Unknown Governmental Organization'},
  'provider': {'id': 211, 'name': 'Norway'},
  'isMobile': False,
  'isMonitor': True,
  'instruments': [{'id': 2, 'name': 'Government Monitor'}],
  'sensors': [{'id': 6253,
    'name': 'no2 µg/m³',
    'parameter': {'id': 5,
     'name': 'no2',
     'units': 'µg/m³',
     'displayName': 'NO₂ mass'}},
   {'id': 6255,
    'name': 'pm10 µg/m³',
    'parameter': {'id': 1,
     'name': 'pm10',
     'units': 'µg/m³',
     'displayName': 'PM10'}},
   {'id': 6254,
    'name': 'pm25 µg/m³',
    'parameter': {'id': 2,
     'name': 'pm25',
     'units': 'µg/m³',
     'displayName': 'PM2.5'}}],
  'coordinates': {'latitude': 60.883643, 'longitude': 11.559301},
  'licenses': None,
  'bounds': [11.559301, 60.883643, 11.559301, 60.883643],
  'distance': None,
  'datetimeFirst': {'utc': '2016-

In [17]:
for location in locations["NO"]:
    print(location["id"], " : ", location["name"])

2947  :  Leiret
3407  :  Lommedalen
3408  :  Øyekast
3546  :  Vollaparken
5362  :  Moheia
5373  :  Rådhuset
5668  :  Grønland
5674  :  Sverresgate
5776  :  Breivoll
6363  :  Alvim
6391  :  Moheia Vest
6502  :  Olav V gate
6916  :  Hønefoss skole
6921  :  Vaterland
6926  :  Åkebergveien
7058  :  Nedre Storgate
7300  :  Ringsakervegen
7317  :  Gartnerløkka
7891  :  Klosterhaugen
7892  :  Bakke kirke
7896  :  St.Croix
7907  :  Haukenes
7917  :  Torvet
7945  :  Vangsveien, Hamar
7972  :  Åsveien skole
7980  :  Kirkeveien
9658  :  Skøyen
10495  :  Alnabru
10498  :  Svanvik
10514  :  Lillehammer barnehage
10523  :  Vårveien
10526  :  Lensmannsdalen
10530  :  Vigernes
10540  :  Rv 4, Aker sykehus
10546  :  Rådal
10550  :  Loddefjord
10554  :  Sofienbergparken
10557  :  Solheim
10563  :  E6-Tiller
10567  :  Kransen
10582  :  Omkjøringsvegen
10591  :  Nygaardsgata
10592  :  Knarrdalstranda
10607  :  Stener Heyerdahl
10608  :  E6 Alna senter
10610  :  Karpdalen
10628  :  Vollgata
10671  :  Holta

get_parameters is used for getting the unit used for measuring respective data

In [ ]:
def get_parameters():
    response = requests.get(endpoint_parameters, headers=headers, params=parameters_params)
    if response.status_code == 200:
        data = response.json()
        para_df = pd.DataFrame(data['results'])
        print("Parameters:")
        print(para_df)
        return para_df
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return pd.DataFrame()

In [ ]:
def get_locations():
    all_cities_with_data = []

    for city in major_us_cities:
        params = {
            "country": "US", "limit": 5, "page": 1
            }
        response = requests.get(endpoint_locations, headers=headers, params=params)
        data = response.json()
    
        if "results" in data:
            city_data = data['results']
            if city_data:
                city_df = pd.DataFrame(city_data)
                city_df['city'] = city
                all_cities_with_data.append(city_df)
            else:
                print(f"No data found for {city}.")

        else:
            # print(f"Error: {response.status_code} - {response.text}")
            pass
    if all_cities_with_data:
        all_cities_df = pd.concat(all_cities_with_data, ignore_index=True)
        print("Locations:")
        print(all_cities_df)
    else:
        print("No locations found for the specified cities.")
        return pd.DataFrame()

In [ ]:
def main():
    get_parameters()
    get_locations()

In [ ]:
if __name__ == "__main__":
    main()

Code not working. Likely due to the fact that JSON for locations uses locality, and not city.